## Visuals

In [49]:
import altair as alt
import altair_saver
import pandas as pd
import shared_utils
from calitp import *
from shared_utils import altair_utils
from shared_utils import styleguide

In [50]:
parquet = pd.read_parquet(
    "gs://calitp-analytics-data/data-analyses/grant_misc/BlackCat_TIRCP_DLA.parquet",
    engine="auto",
)

In [51]:
parquet.head()

,locode,name,BC_TIRCP_DLA_merge
0,6302.0,Humboldt Bay Harbor Recreation & Conservation ...,DLA Only
1,6330.0,Willow Creek Community Services District,DLA Only
2,5036.0,Trinidad,DLA Only
3,5049.0,Ukiah,DLA Only
4,5082.0,Willits,DLA Only


In [52]:
#Aggregating
agg = (
    parquet.drop_duplicates()
    .groupby(["BC_TIRCP_DLA_merge", "name"])
    .agg({"locode": "max"})
    .drop( columns = ['locode']) 
)

In [53]:
agg

Empty DataFrame
Columns: []
Index: [(BlackCat Only, ARC Bakersfield), (BlackCat Only, ARC Imperial Valley), (BlackCat Only, Able Industries), (BlackCat Only, Alegria Community Living), (BlackCat Only, Alpine County Community Development), (BlackCat Only, Amador Transit), (BlackCat Only, Angel View, Inc.), (BlackCat Only, Area 1 Agency on Aging), (BlackCat Only, Asian Community Center of Sacramento Valley, Inc. DBA ACC Senior Services), (BlackCat Only, Avenidas ), (BlackCat Only, Bay Area Outreach & Recreation Program, Inc), (BlackCat Only, Big Valley 50 Plus), (BlackCat Only, Butte County Association of Governments/ Butte Regional Transit), (BlackCat Only, Calaveras County Public Works), (BlackCat Only, Calaveras Transit Agency ), (BlackCat Only, Care-A-Van Transit Systems, Inc.), (BlackCat Only, Casa Allegra Community Services), (BlackCat Only, Catholic Charities of the Diocese of Stockton), (BlackCat Only, Center for Elders' Independence ), (BlackCat Only, Choice in Aging), (BlackCat Only, City of Arcata), (BlackCat Only, City of Arvin), (BlackCat Only, City of Auburn), (BlackCat Only, City of California City), (BlackCat Only, City of Chowchilla ), (BlackCat Only, City of Corcoran - Corcoran Area Transit), (BlackCat Only, City of Dinuba ), (BlackCat Only, City of Dixon), (BlackCat Only, City of Escalon ), (BlackCat Only, City of Fairfield), (BlackCat Only, City of Fontana), (BlackCat Only, City of Guadalupe), (BlackCat Only, City of Lafayette: Lamorinda Sprit Van Program), (BlackCat Only, City of McFarland), (BlackCat Only, City of Needles), (BlackCat Only, City of Ojai), (BlackCat Only, City of Petaluma), (BlackCat Only, City of Porterville), (BlackCat Only, City of Ridgecrest), (BlackCat Only, City of Rio Vista), (BlackCat Only, City of Roseville), (BlackCat Only, City of Santa Maria), (BlackCat Only, City of Santa Rosa), (BlackCat Only, City of Shafter), (BlackCat Only, City of Solvang), (BlackCat Only, City of Taft), (BlackCat Only, City of Tehachapi), (BlackCat Only, City of Visalia), (BlackCat Only, City of Vista ), (BlackCat Only, City of Wasco), (BlackCat Only, City of Woodlake), (BlackCat Only, Coastal Seniors, Inc.), (BlackCat Only, Colusa County Transit Agency), (BlackCat Only, Common Ground Senior Services), (BlackCat Only, Community Bridges/Liftline), (BlackCat Only, Community Partners In Caring), (BlackCat Only, Consolidated Tribal Health Project, Inc.), (BlackCat Only, Contra Costa ARC), (BlackCat Only, County Connection), (BlackCat Only, County of Los Angeles - Department of Public Works), (BlackCat Only, County of Mariposa), (BlackCat Only, County of Nevada Public Works, Transit Services Division), (BlackCat Only, County of Sacramento Department of Child, Family and Adult Services), (BlackCat Only, County of Sacramento Department of Transportation), (BlackCat Only, County of Shasta Department of Public Works), (BlackCat Only, County of Siskiyou), (BlackCat Only, County of Sonoma, Human Services Department, Adult and Aging Division), (BlackCat Only, County of Tulare), (BlackCat Only, County of Ventura, Health Services Agency), (BlackCat Only, Delano Association for the Developmentally Disabled, Inc.), (BlackCat Only, Desert ARC), (BlackCat Only, Desert Access and Mobility, Inc), (BlackCat Only, Desert Area Resources and Training), (BlackCat Only, Dignity Community Care), (BlackCat Only, Dignity Health Connected Living), (BlackCat Only, Drivers for Survivors, Inc.), (BlackCat Only, East Bay Services to the Developmentally Disabled ), (BlackCat Only, Eastern Contra Costa Transit Authority), (BlackCat Only, Eastern Sierra Transit Authority), (BlackCat Only, Easy Lift Transportation, Inc.), (BlackCat Only, Eden I&R, INC.), (BlackCat Only, ElderHelp of San Diego), (BlackCat Only, Eureka Transit Service), (BlackCat Only, Faith in Action), (BlackCat Only, Family Bridges, Inc.), (BlackCat Only, Foothill AIDS Project), (BlackCat Only, Friends of Adult Day Health Care Centers), (BlackCat Only, Friends of Children with 

In [54]:
with pd.ExcelWriter("./Master_Agreement_Overlaps.xlsx") as writer:
    agg.to_excel(writer, sheet_name="Organizations", index=True)

In [55]:
parquet.name.nunique()

1217

In [56]:
summary = (
    parquet.BC_TIRCP_DLA_merge.value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={"index": "Grants",
                    "BC_TIRCP_DLA_merge": "Count_of_Organizations"})
)

In [57]:
summary

,Grants,Count_of_Organizations
0,DLA Only,991
1,BlackCat Only,168
2,BlackCat and DLA,33
3,TIRCP_Only,21
4,"TIRCP, BlackCat and DLA",17
5,TIRCP and BlackCat,1


In [58]:
legend = ['DLA Only', 'BlackCat Only', 'BlackCat and DLA', 'TIRCP Only', 'TIRCP, BlackCat, and DLA', 'TIRCP and BlackCat']

In [59]:
# Advanced bar chart: label the bars & re order the legend.
def labeling(word):
    # Add specific use cases where it's not just first letter capitalized
    LABEL_DICT = { "prepared_y": "Year",
              "dist": "District",
              "nunique":"Number of Unique",
              "project_no": "Project Number"}
    
    if (word == "mpo") or (word == "rtpa"):
        word = word.upper()
    elif word in LABEL_DICT.keys():
        word = LABEL_DICT[word]
    else:
        #word = word.replace('n_', 'Number of ').title()
        word = word.replace('unique_', "Number of Unique ").title()
        word = word.replace('_', ' ').title()
    
    return word

def base_bar(df):
    chart = alt.Chart(df).mark_bar()
    return chart


# Function
def make_bar(
    df, y_col, x_col, label_col, chart_title=""
):  # label_col is the column I want the bar graph to be labeled by.

    if chart_title == "":
        chart_title = (
            f"{labeling(x_col)} by {labeling(y_col)}"  # Allows me to customize title.
        )

    bar = base_bar(df)

    bar = bar.encode(
        x=alt.X(x_col, title=labeling(x_col)),
        y=alt.Y(
            y_col, title=labeling(y_col), sort=("-x")
        ),  # Sorts chart with X column from descending.
        color=alt.Color(
            y_col,
            scale=alt.Scale(
                domain=legend,  # List that specifies the order of the legend.
                range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS,
            ),
        ),
    )
    # https://stackoverflow.com/questions/54015250/altair-setting-constant-label-color-for-bar-chart
    text = bar.mark_text(align="left", baseline="middle", color="black", dy=3).encode(
        text=label_col,
        # Set color here, because encoding for mark_text gets
        # superseded by alt.Color
        color=alt.value("black"),
    )

    chart = bar + text

    chart = (
        styleguide.preset_chart_config(chart)
        .properties(title=chart_title)
        .configure_axis(grid=False)
    )

    display(chart)

In [60]:
make_bar(summary, 'Grants','Count_of_Organizations','Count_of_Organizations', '# of Organizations by Grant')

alt.LayerChart(...)